In [11]:
import numpy as np
import os
from urllib import request
import gzip
import pickle
import data
from sklearn.model_selection import train_test_split
import time
import matplotlib.pyplot as plt

In [12]:
#data.init()

In [13]:
def load_mnist(final=False, flatten=False):
    """
    Load the MNIST data
    :param final: If true, return the canonical test/train split. If false, split some validation data from the training
       data and keep the test data hidden.
    :param flatten:
    :return:
    """

    if not os.path.isfile('mnist.pkl'):
        init()

    xtrain, ytrain, xtest, ytest = load()
    xtl, xsl = xtrain.shape[0], xtest.shape[0]

    if flatten:
        xtrain = xtrain.reshape(xtl, -1)
        xtest  = xtest.reshape(xsl, -1)

    if not final: # return the flattened images
        return (xtrain[:-5000], ytrain[:-5000]), (xtrain[-5000:], ytrain[-5000:]), 10

    return (xtrain, ytrain), (xtest, ytest), 10

# Numpy-only MNIST loader. Courtesy of Hyeonseok Jung
# https://github.com/hsjeong5/MNIST-for-Numpy

filename = [
["training_images","train-images-idx3-ubyte.gz"],
["test_images","t10k-images-idx3-ubyte.gz"],
["training_labels","train-labels-idx1-ubyte.gz"],
["test_labels","t10k-labels-idx1-ubyte.gz"]
]

def download_mnist():
    base_url = "http://yann.lecun.com/exdb/mnist/"
    for name in filename:
        print("Downloading "+name[1]+"...")
        request.urlretrieve(base_url+name[1], name[1])
    print("Download complete.")

def save_mnist():
    mnist = {}
    for name in filename[:2]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=16).reshape(-1,28*28)
    for name in filename[-2:]:
        with gzip.open(name[1], 'rb') as f:
            mnist[name[0]] = np.frombuffer(f.read(), np.uint8, offset=8)
    with open("mnist.pkl", 'wb') as f:
        pickle.dump(mnist,f)
    print("Save complete.")

def init():
    download_mnist()
    save_mnist()

def load():
    with open("mnist.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [14]:
(xtrain, ytrain), (xtest, ytest), digits = load_mnist()

xtrain = (xtrain/255).astype('float32')
xtest = (xtest/255).astype('float32')

In [15]:
#Train Set
onehot_y = np.zeros((ytrain.size, ytrain.max()+1))
onehot_y[np.arange(ytrain.size),ytrain] = 1
# Test Set
onehot_y_test = np.zeros((ytest.size, ytest.max()+1))
onehot_y_test[np.arange(ytest.size),ytest] = 1

ytrain = onehot_y
ytest = onehot_y_test

In [5]:
print(xtrain.shape)
print(ytrain.shape)
print(xtest.shape)
print(ytest.shape)
print(digits)

(55000, 784)
(55000, 10)
(5000, 784)
(5000, 10)
10


In [19]:
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [20]:
class DeepNeuralNetwork():
    def __init__(self, xtrain, sizes, epochs=5, l_rate=0.001):
        self.sizes = sizes
        self.epochs = epochs
        self.records = xtrain.shape[0]
        self.l_rate = l_rate

        # we save all parameters in the neural network in this dictionary
        self.params = self.initialization()

    def sigmoid(self, x, derivative=False):
        if derivative:
            return (np.exp(-x))/((np.exp(-x)+1)**2)
        return 1/(1 + np.exp(-x))

    def softmax(self, x, derivative=False):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        if derivative:
            return exps / np.sum(exps, axis=0) * (1 - exps / np.sum(exps, axis=0))
        return exps / np.sum(exps, axis=0)

    def initialization(self):
        # number of nodes in each layer
        input_layer=self.sizes[0]
        hidden_1=self.sizes[1]
        output_layer=self.sizes[2]
        np.random.seed(2)

        params = { 
            'W1':np.random.randn(input_layer,hidden_1) * np.sqrt(1. / hidden_1),
            'W2':np.random.randn(hidden_1,output_layer) * np.sqrt(1. / output_layer),
            'b1': np.zeros((1,300)),
            'b2': np.zeros((1,10))
        }
        
        return params

    def forward_pass(self, x_train):
        params = self.params
        
        W1 = params["W1"]
        b1 = params["b1"]
        W2 = params["W2"]
        b2 = params["b2"]


        # input layer activations becomes sample
        params['A0'] = x_train
        # input layer to hidden layer 1
        params['Z1'] = np.dot(W1.T,params['A0']) + params['b1']
        params['A1'] = self.sigmoid(params['Z1'])
        # hidden layer 1 to output layer
        params['Z2'] = np.dot(params['A1'],params["W2"]) + params['b2']
        params['A2'] = self.softmax(params['Z2'])

        return params['A2']

    def backward_pass(self, y_train, output):
           
        params = self.params
        change_w = {}
        
        #print(params.keys())
        A2 = params['A2']
        A1 = params['A1']
        A0 = params['A0']
        Z2 = params['Z2']
        Z1 = params['Z1']
        W2 = params['W2']
        W1 = params['W1']
        b2 = params['b2']
        b1 = params['b1']
        
        
        dZ2 = A2-y_train
        dW2 = dZ2 * A1.T
        db2 = np.sum(dZ2, axis = 1, keepdims = True)
        dZ1 = np.dot(dZ2, W2.T) * (A1*(1-A1))
        dW1 = dZ1.T * A0.T
        dW1 = dW1.T
        db1 = np.sum(dZ1, axis = 1, keepdims = True)
        
        change_w['dZ2'] = dZ2
        change_w['dW2'] = dW2
        change_w['db2'] = db2
        change_w['dZ1'] = dZ1
        change_w['dW1'] = dW1
        change_w['db1'] = db1

        return change_w

    def update_network_parameters(self, changes_to_w):
        
        W1 = self.params["W1"]
        b1 = self.params["b1"]
        W2 = self.params["W2"]
        b2 = self.params["b2"]
    
        dW1 = changes_to_w["dW1"]
        db1 = changes_to_w["db1"]
        dW2 = changes_to_w["dW2"]
        db2 = changes_to_w["db2"]

        W1 = W1 - dW1 * self.l_rate
        b1 = b1 - db1 * self.l_rate
        W2 = W2 - dW2 * self.l_rate
        b2 = b2 - db2 * self.l_rate
        
        self.params['W1'] = W1
        self.params['W2'] = W2
        self.params['b1'] = b1
        self.params['b2'] = b2
        
    
    def compute_cost(self, A2, Y):

        m = Y.shape[0] # number of example

        cost = -1/ m*np.sum(np.multiply(np.log(A2),Y) + np.multiply(np.log(1-A2), (1-Y)))

        cost = np.squeeze(cost)    
        assert(isinstance(cost, float))

        return cost
    
    
    def compute_accuracy(self, x_val, y_val):
        
        predictions = []

        for x, y in zip(x_val, y_val):
            output = self.forward_pass(x)
            pred = np.argmax(output)
            predictions.append(pred == np.argmax(y))
        
        return np.mean(predictions)
    
    
    def train(self, x_train, y_train, x_val, y_val):
        
        train_loss = list()
        validation_loss = list()
        train_accuracy = list()
        validation_accurracy =  list()
        start_time = time.time()
        m = x_train.shape[0]
        n = x_val.shape[0]
        
        for iteration in range(self.epochs):
            loss_train = 0
            for x,y in zip(x_train, y_train):
                output = self.forward_pass(x)
                changes_to_w = self.backward_pass(y, output)
                self.update_network_parameters(changes_to_w) 
                loss_train = loss_train + self.compute_cost(output,y)
            cost_train = loss_train/m
            train_loss.append(cost_train)
            
            loss_val = 0
            for x,y in zip(x_val, y_val):
                output = self.forward_pass(x)
                changes_to_w = self.backward_pass(y, output)
                self.update_network_parameters(changes_to_w) 
                loss_val = loss_val + self.compute_cost(output,y)   
            cost_val = loss_val/n
            validation_loss.append(cost_val)
            
            accuracy = self.compute_accuracy(x_train, y_train)
            train_accuracy.append(accuracy*100)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                iteration+1, time.time() - start_time, accuracy * 100))
            
            accuracy = self.compute_accuracy(x_val, y_val)
            validation_accurracy.append(accuracy*100)
            print('Epoch: {0}, Time Spent: {1:.2f}s, Accuracy: {2:.2f}%'.format(
                iteration+1, time.time() - start_time, accuracy * 100))
       
        return train_loss, validation_loss, train_accuracy, validation_accurracy

In [ ]:
dnn = DeepNeuralNetwork(xtrain,sizes=[784, 300, 10], l_rate=0.05)
train_loss, valid_loss, train_accuracy, valid_accuracy =  dnn.train(xtrain, ytrain, xtest, ytest)


Epoch: 1, Time Spent: 172.45s, Accuracy: 9.88%
Epoch: 1, Time Spent: 173.16s, Accuracy: 9.78%
Epoch: 2, Time Spent: 375.97s, Accuracy: 9.88%
Epoch: 2, Time Spent: 376.75s, Accuracy: 9.78%


###  Code for Plotting

In [123]:
def training_validation_loss(list_loss_train, list_loss_validation):
        # Create count of the number of epochs
        epoch_count = range(1, len(list_loss_train) + 1)
        
        #plotting
        plt.plot(epoch_count,list_loss_train,"-b", label=" training")
        plt.plot(epoch_count,list_loss_validation, "-r", label="validation")
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.xticks(rotation=60)
        plt.title('Loss w.r.t. epoch for learning rate = 0.05')
        plt.legend(["Training", "Validation"], loc ="upper right") 
        plt.show()

In [124]:
def training_validation_accuracy(list_train_accuracy,list_valid_accuracy):
        # Create count of the number of epochs
        epoch_count = range(1, len(list_train_accuracy) + 1)
        
        #plotting
        plt.plot(epoch_count,list_train_accuracy,"-b", label=" training")
        plt.plot(epoch_count,list_valid_accuracy, "-r", label="validation")
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.xticks(rotation=60)
        plt.title('Accuracy w.r.t. epoch for learning rate = 0.05')
        plt.legend(["Training", "Validation"], loc ="upper right") 
        plt.show()

In [125]:
## Plot for training vs validation  loss per epoch
dnn = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.05)
train_loss, valid_loss, train_accuracy, valid_accuracy =  dnn.train(xtrain, ytrain, xtest, ytest)
training_validation_loss(train_loss,valid_loss)
training_validation_accuracy(train_accuracy,valid_accuracy)

In [126]:
def sgd_learning_rates(valid_loss_1,valid_loss_2,valid_loss_3):
    epoch_count = range(1, len(valid_loss_1) + 1)
    plt.plot(epoch_count,valid_loss_1,"-b", label=" lr_0.05") 
    plt.plot(epoch_count,valid_loss_2,"-r", label=" lr_0.01") 
    plt.plot(epoch_count,valid_loss_3,"-g", label=" lr_0.0001") 
    plt.ylabel('Performance (Loss)')
    plt.xlabel('Epoch')
    plt.xticks(rotation=60)
    plt.title('Loss over 5 epoches')
    plt.legend(["lr_0.05", "lr_0.01","lr_0.0001"], loc ="upper right") 
    plt.show()  

In [127]:
## Plot for SGD with different learning rates

dnn_1 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.05)
dnn_2 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.01)
dnn_3 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.0001)
train_loss_1, valid_loss_1, train_accuracy_1, valid_accuracy_1 = dnn_1.train(xtrain, ytrain, xtest, ytest)
train_loss_2, valid_loss_2, train_accuracy_2, valid_accuracy_2 = dnn_2.train(xtrain, ytrain, xtest, ytest)
train_loss_3, valid_loss_3, train_accuracy_3, valid_accuracy_3 = dnn_3.train(xtrain, ytrain, xtest, ytest)

In [129]:
sgd_learning_rates(valid_loss_1,valid_loss_2,valid_loss_3)

In [130]:
dnn_1 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.001)
dnn_2 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.001)
dnn_3 = DeepNeuralNetwork(sizes=[784, 300, 10], l_rate=0.001)
train_loss_1, valid_loss_1, train_accuracy_1, valid_accuracy_1 = dnn_1.train(xtrain, ytrain, xtest, ytest)
train_loss_2, valid_loss_2, train_accuracy_2, valid_accuracy_2 = dnn_2.train(xtrain, ytrain, xtest, ytest)
train_loss_3, valid_loss_3, train_accuracy_3, valid_accuracy_3 = dnn_3.train(xtrain, ytrain, xtest, ytest)

In [ ]:
import statistics
# Plot with mean and std. deviation
epoch_count = range(1, len(valid_loss_1) + 1)
means= list()
std_deviation = list()
temp = list()

for i in range(len(epoch_count)):
    temp.append(valid_loss_1[i])
    temp.append(valid_loss_2[i])
    temp.append(valid_loss_3[i])
    means.append(statistics.mean(temp))


for i in range(len(epoch_count)):
    temp.append(valid_loss_1[i])
    temp.append(valid_loss_2[i])
    temp.append(valid_loss_3[i])
    std_deviation.append(statistics.stdev(temp))


# plot it!
fig, ax = plt.subplots(1)
ax.plot(epoch_count, means + std_deviation , lw=2, label='mean+SGD', color='blue')
ax.plot(epoch_count, means - std_deviation , lw=2, label='mean-SGD', color='orange')
ax.plot(epoch_count, means , lw=2, label='green', color='blue')
ax.set_title('Average and Std. deviation of Loss')
ax.legend(loc='upper left')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.grid()
